In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import kendalltau
from tqdm.notebook import tqdm
from IPython.display import clear_output
import os, shutil, gc
import torch

REF_MODELS = [
    "Yi-1.5-9B-Chat",
    "Meta-Llama-3.1-8B-Instruct",
    "Phi-3-medium-128k-instruct",
    "Mistral-7B-Instruct-v0.3",
    "Qwen2.5-0.5B-Instruct",
    "Qwen2.5-1.5B-Instruct",
    "Qwen3-4B-Instruct-2507"
    ]

# Renaming Filenames for Compatibility

In [ ]:
# renaming Aymen's files for better f-string-types.
fnames = sorted([f for f in os.listdir("raw_results/aymen") if ".csv" in f])
for fname in fnames:

    # get the new filename
    new_fname = fname.replace("training_logs_", "").replace("arwc_normalized", "Original")\
    .replace("offline_1", "VDW").replace("offline_2", "VAW")\
    .replace("online_1", "SWCW").replace("online_2", "TSW")

    # save these files
    os.rename(f"raw_results/aymen/{fname}", f"raw_results/aymen/{new_fname}")

In [ ]:
# renaming Skyler's files for better f-string-types
fnames = sorted([f for f in os.listdir("raw_results/skyler") if ".csv" in f])

for fname in fnames:

    # get our new filename
    new_fname = fname.replace("PKU-SafeRLHF-30K-standard", "SafeRLHF").replace("ultrafeedback_binarized", "UltraFeedback")\
    .replace("Offline-1-One-Hot", "SWCW-OH").replace("Thompson-DPO", "TSW").replace("baseline-DPO", "DPO")\
    .replace("01-ai_Yi-1.5-9B-Chat", "Yi-1.5-9B").replace("Qwen_Qwen2.5-0.5B-Instruct", "Qwen2.5-0.5B")\
    .replace("Qwen_Qwen2.5-1.5B-Instruct", "Qwen2.5-1.5B")\
    .replace("Qwen_Qwen3-4B-Instruct-2507", "Qwen3-4B")\
    .replace("meta-llama_Meta-Llama-3.1-8B-Instruct", "Llama-3.1-8B")\
    .replace("microsoft_Phi-3-medium-128k-instruct", "Phi-3-Medium-128k")\
    .replace("mistralai_Mistral-7B-Instruct-v0.3", "Mistral-7B")

    # save these files
    os.rename(f"raw_results/skyler/{fname}", f"raw_results/skyler/{new_fname}")

# Summarizing Relevant Metrics Per Method

In [ ]:
'''
List of Methods:
DPO - 7x reference models (FINISHED)
Original - MRPO/MDPO: 2x.
VDW - MRPO/MDPO: 2x ("Offline 1") (FINISHED)
VAW - MRPO/MDPO: 2x ("Offline 2") (FINISHED)
SWCW - MRPO/MDPO: 2x ("Online 1") (FINISHED)
SWCW-OH - MRPO/MDPO: 2x ("Online 1 (0-1)") (FINISHED)
TSW - 1x ("Online 2") (FINISHED)
'''
# create summary logs for DPO: final test acc, Delta (batch-acc-pre, post) (<0, =0, >0)
dpo_summary = pd.DataFrame(
    data=None, columns=[
        "method", "dataset", "reference", "seed", "final_test_acc",
        "prop_delta_neg", "prop_delta_zero", "prop_delta_pos"])

# fill out these logs for DPO.
dpo_fnames = sorted([f for f in os.listdir("raw_results/skyler") if "DPO" in f])
for fname in dpo_fnames:

    # record our settings
    method = "DPO"
    dataset = fname.split("_")[1].split("=")[1]
    reference = fname.split("_")[2].split("=")[1]
    seed = int(fname.split("_")[3].split("=")[1].split(".csv")[0])

    # load in our dataframe
    df = pd.read_csv(f"raw_results/skyler/{fname}")
    final_test_acc = df.test_acc.values[-1]

    # compute our delta statistics
    deltas = (df.batch_acc_post - df.batch_acc_pre)
    prop_delta_neg = (deltas < 0).mean()
    prop_delta_zero = (deltas == 0).mean()
    prop_delta_pos = (deltas > 0).mean()

    # assemble row + add to our dataframe
    row = [method, dataset, reference, seed, final_test_acc,
           prop_delta_neg, prop_delta_zero, prop_delta_pos]
    dpo_summary.loc[len(dpo_summary.index)] = row

# save to our cleaned directory
dpo_summary.to_csv("cleaned_results/dpo_summary.csv", index=False)

In [ ]:
'''
Metrics of Interest:
1. final test acc, Delta (batch-acc-pre, post) (<0, =0, >0)
2. mu_is (i=0, to i=6) - i.e., posterior reward means per arm
'''
# create summary logs for Thompson Sampling
tsw_summary = pd.DataFrame(
    data=None, columns=[
        "method", "dataset", "piv", "seed", "final_test_acc",
        "prop_delta_neg", "prop_delta_zero", "prop_delta_pos"]\
    + [f"mu_{i}" for i in range(7)])

# fill out these logs for Thomspon Sampling
tsw_fnames = sorted([f for f in os.listdir("raw_results/skyler") if "TSW" in f])
for fname in tsw_fnames:

    # record our settings
    method = "TSW"
    dataset = fname.split("_")[1].split("=")[1]
    piv = float(fname.split("_")[2].split("=")[1])
    seed = int(fname.split("_")[3].split("=")[1].split(".csv")[0])

    # load in our dataframe
    df = pd.read_csv(f"raw_results/skyler/{fname}")
    final_test_acc = df.test_acc.values[-1]

    # compute our delta statistics
    deltas = (df.batch_acc_post - df.batch_acc_pre)
    prop_delta_neg = (deltas < 0).mean()
    prop_delta_zero = (deltas == 0).mean()
    prop_delta_pos = (deltas > 0).mean()

    # get our Beta hyperparameters
    a = df[[f"a{i}" for i in range(7)]].values
    b = df[[f"b{i}" for i in range(7)]].values
    mus = (a / (a+b))[-1,:]

    # assemble row + add to our dataframe
    row = [method, dataset, piv, seed, final_test_acc,
           prop_delta_neg, prop_delta_zero, prop_delta_pos] + list(mus)
    tsw_summary.loc[len(tsw_summary.index)] = row

# save to our cleaned directory
tsw_summary.to_csv("cleaned_results/tsw_summary.csv", index=False)

In [ ]:
'''
Offline Variants:
VDW - MRPO/MDPO: 2x ("Offline 1")
VAW - MRPO/MDPO: 2x ("Offline 2")
'''
# create summary logs for Offline 1 + Offline 2
offlines_summary = pd.DataFrame(
    data=None, columns=[
        "method", "dataset", "base", "seed", "final_test_acc",
        "prop_delta_neg", "prop_delta_zero", "prop_delta_pos"]\
    + [f"alpha_{i}" for i in range(7)])

# fill out these logs for Offline Methods
offlines_fnames = sorted([f for f in os.listdir("raw_results/aymen") if "VAW" in f or "VDW" in f])
for fname in offlines_fnames:

    # extract out our settings
    dataset, method, seed, base = fname.replace(".csv", "").split("_"); seed = int(seed)

    # load in our dataframe
    df = pd.read_csv(f"raw_results/aymen/{fname}")
    final_test_acc = float(df.test_acc.values[-1].split(", device")[0].split("tensor(")[1])

    # compute our delta statistics
    deltas = (df.batch_acc_post - df.batch_acc_pre)
    prop_delta_neg = (deltas < 0).mean()
    prop_delta_zero = (deltas == 0).mean()
    prop_delta_pos = (deltas > 0).mean()

    # also extract our alphas
    alphas = df[[col for col in df.columns if "alpha" in col]].values[-1]

    # assemble row + add to our dataframe
    row = [method, dataset, base, seed, final_test_acc,
           prop_delta_neg, prop_delta_zero, prop_delta_pos] + list(alphas)
    offlines_summary.loc[len(offlines_summary.index)] = row

# save to our cleaned directory
offlines_summary.to_csv("cleaned_results/offlines_summary.csv", index=False)

In [ ]:
'''
Online 1 Variants:
SWCW - MRPO/MDPO: 2x ("Online 1")
SWCW-OH - MRPO/MDPO: 2x ("Online 1 (0-1)")
'''
# create summary logs for the Online 1 variants.
swcw_summary = pd.DataFrame(
    data=None, columns=[
        "method", "dataset", "base", "seed", "one_hot", "final_test_acc",
        "prop_delta_neg", "prop_delta_zero", "prop_delta_pos"])

# fill out these logs for Aymen's original SWCW runs first.
swcw_fnames = sorted([f for f in os.listdir("raw_results/aymen") if "SWCW" in f])
for fname in swcw_fnames:

    # extract out our settings
    dataset, method, seed, base = fname.replace(".csv", "").split("_"); seed = int(seed)

    # load in our dataframe
    df = pd.read_csv(f"raw_results/aymen/{fname}")
    final_test_acc = float(df.test_acc.values[-1].split(", device")[0].split("tensor(")[1])

    # compute our delta statistics
    deltas = (df.batch_acc_post - df.batch_acc_pre)
    prop_delta_neg = (deltas < 0).mean()
    prop_delta_zero = (deltas == 0).mean()
    prop_delta_pos = (deltas > 0).mean()

    # assemble row + add to our dataframe
    row = [method, dataset, base, seed, False, final_test_acc,
           prop_delta_neg, prop_delta_zero, prop_delta_pos]
    swcw_summary.loc[len(swcw_summary.index)] = row

# now work on Skyler's SWCW-OH runs next.
swcw_fnames = sorted([f for f in os.listdir("raw_results/skyler") if "SWCW" in f])
for fname in swcw_fnames:

    # extract out our settings
    dataset = fname.split("_")[1].split("=")[1]
    method = "SWCW"
    seed = int(fname.split("_")[2].replace(".csv", "").split("=")[1])
    base = "MRPO"

    # load in our dataframe
    df = pd.read_csv(f"raw_results/skyler/{fname}")
    final_test_acc = df.test_acc.values[-1]

    # compute our delta statistics
    deltas = (df.batch_acc_post - df.batch_acc_pre)
    prop_delta_neg = (deltas < 0).mean()
    prop_delta_zero = (deltas == 0).mean()
    prop_delta_pos = (deltas > 0).mean()

    # assemble row + add to our dataframe
    row = [method, dataset, base, seed, True, final_test_acc,
           prop_delta_neg, prop_delta_zero, prop_delta_pos]
    swcw_summary.loc[len(swcw_summary.index)] = row

# save to our cleaned directory
swcw_summary.to_csv("cleaned_results/online-1s_summary.csv", index=False)

In [ ]:
'''
Original - MRPO/MDPO: 2x.
'''
# create summary logs for Offline 1 + Offline 2
originals_summary = pd.DataFrame(
    data=None, columns=[
        "method", "dataset", "base", "seed", "final_test_acc",
        "prop_delta_neg", "prop_delta_zero", "prop_delta_pos"])

# fill out these logs for Le et al.'s original method.
originals_fnames = sorted([f for f in os.listdir("raw_results/aymen") if "Original" in f])
for fname in originals_fnames:

    # extract out our settings
    dataset, method, seed, base = fname.replace(".csv", "").split("_"); seed = int(seed)

    # load in our dataframe
    df = pd.read_csv(f"raw_results/aymen/{fname}")
    final_test_acc = float(df.test_acc.values[-1].split(", device")[0].split("tensor(")[1])

    # compute our delta statistics
    deltas = (df.batch_acc_post - df.batch_acc_pre)
    prop_delta_neg = (deltas < 0).mean()
    prop_delta_zero = (deltas == 0).mean()
    prop_delta_pos = (deltas > 0).mean()

    # assemble row + add to our dataframe
    row = [method, dataset, base, seed, final_test_acc,
           prop_delta_neg, prop_delta_zero, prop_delta_pos]
    originals_summary.loc[len(originals_summary.index)] = row

# save to our cleaned directory
originals_summary.to_csv("cleaned_results/originals_summary.csv", index=False)